In [ ]:
from matplotlib import pyplot as plt
from matplotlib import patches
import numpy as np
import librosa
import sounddevice as sd
import soundfile as sf
import IPython.display as ipd
import librosa.display
import scipy
import sys
import pandas as pd

In [ ]:
class EnvironmentData:
    def __init__(self, csvFile):
        self.data = pd.read_csv(csvFile)

In [ ]:
csvFile = './data/tasmax_timeseries_monthly_cru_1901-2021_GBR.csv'

environmentDataOfUK = EnvironmentData(csvFile)

